In [1]:
import os
import pandas as pd
import requests
from datetime import datetime
import json
from collections import OrderedDict

In [2]:
%run common.ipynb

In [3]:
%%time

# 读取2000年至2013年的每一天的 S&P 股票的资料,如果有股市开的话

# url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=MSFT&outputsize=full&apikey=demo'
# web_data = requests.get(url)

API_KEY = "25IEBEKH83S6MQOE"
page_num = 1
page_size = 100
url_params = {"function":"TIME_SERIES_DAILY_ADJUSTED", "symbol": ".INX", "outputsize": "full", "apikey": API_KEY}
everything_url="https://www.alphavantage.co/query?"
web_data = requests.get(everything_url, params=url_params)

web_data.encoding = 'utf-8'
web_data_text = web_data.text
json_acceptable_string = web_data_text.replace("'", "\"")
web_data_dict = json.loads(json_acceptable_string)

print(type(web_data_dict), len(web_data_dict), web_data_dict.keys())

del web_data, web_data_text, json_acceptable_string

meta_data_dict = web_data_dict["Meta Data"]
stock_data_dict = web_data_dict["Time Series (Daily)"]

del web_data_dict

print(meta_data_dict, "\n")
print(len(stock_data_dict))

# print(type(stock_data_dict), stock_data_dict.keys())

# stock_time = list(stock_data_dict.keys())
# print(type(stock_time), stock_time[:3])

# stock_val = stock_data_dict.values()
# term = stock_data_dict["2018-05-24"]
# print(type(term), term, term["4. close"])

<class 'dict'> 2 dict_keys(['Meta Data', 'Time Series (Daily)'])
{'1. Information': 'Daily Time Series with Splits and Dividend Events', '5. Time Zone': 'US/Eastern', '3. Last Refreshed': '2018-06-20', '2. Symbol': 'S&P500 Index', '4. Output Size': 'Full size'} 

4646
CPU times: user 57.1 ms, sys: 47.5 ms, total: 105 ms
Wall time: 3.44 s


In [4]:
# 把获取到的股票data按日期排序,并按照今天比昨天是否增长设置1和0
# 今天比昨天价格更低为0,否则为1(可能是涨,可能是维持不变)
ostock_data_dict = OrderedDict(sorted(stock_data_dict.items()))

new_stocks = dict()
new_stock_keys = list(ostock_data_dict.keys())
print(type(new_stock_keys))
old_key = new_stock_keys[0]
for now_key in new_stock_keys:
    if ostock_data_dict[now_key]["4. close"] < ostock_data_dict[old_key]["4. close"]:
        new_stocks[now_key] = 0
    else:
        new_stocks[now_key] = 1
    old_key = now_key
    
# Print data for checking
print(stock_data_dict["2000-01-03"], stock_data_dict["2000-01-03"]["4. close"])
print(len(new_stocks), len(ostock_data_dict))
print(new_stocks["2000-01-03"], stock_data_dict["2000-01-03"]["4. close"]) # first day from API
print(new_stocks["2000-01-04"], stock_data_dict["2000-01-04"]["4. close"])
print(new_stocks["2000-01-05"], stock_data_dict["2000-01-05"]["4. close"])
print(new_stocks["2006-10-20"], stock_data_dict["2006-10-20"]["4. close"]) # first day of news dataset
print(new_stocks["2010-12-31"], stock_data_dict["2010-12-31"]["4. close"]) # last day of news dataset

<class 'list'>
{'4. close': '1455.2200', '7. dividend amount': '0.0000', '6. volume': '931800000', '3. low': '1438.3600', '5. adjusted close': '1455.2200', '1. open': '1469.2500', '2. high': '1478.0000', '8. split coefficient': '1.0000'} 1455.2200
4646 4646
1 1455.2200
0 1399.4200
1 1402.1100
1 1368.6000
0 1257.6400


In [5]:
# Make sure news dataset has been extracted (follow instructions on README.md)
TRAINING_PATH = './data/news/'
TESTING_PATH = './data/news/'

In [6]:
old_categories = [dirname for dirname in os.listdir(TRAINING_PATH)]
categories= sorted(old_categories)
print("Number of categories: %d; first=%s, last=%s" % (len(categories), categories[0], categories[-1]))

Number of categories: 1266; first=2010-05-03, last=2013-11-26


In [10]:
%%time

# Read News Headlines
train_list = []
skip_count = 0

for category in categories:
    category_path = TRAINING_PATH + category + "/"
    print(category_path)
    for filename in os.listdir(category_path):
        filepath = category_path + filename
        
        with open(filepath, encoding='utf-8') as file:
            try:
                words = file.read().strip().split('-- http://www.bloomberg.com')
                if words[0] == "":
                    pass
                else:
                    abs_words = words[0].split("\n")
                    if len(abs_words) > 2:
                        # Remove '-- ' prefix of time and title
                        ari_title = abs_words[0].replace('-- ', '')
                        ari_time = abs_words[2].replace('-- ', '')[:10]

                        # Check if ari_time is valid
                        if utcdate2timestamp(ari_time[0:10]) == False:
                            skip_count += 1
                            continue
                        if category in stock_data_dict.keys() and len(ari_title) > 0:
                            price = new_stocks[category]
                        else:
                            price = 1 # 表示這一天沒有股票價格信息，股票價格不變
                    
                        train_list.append([ari_time, ari_title, price])
            except ValueError as e:
                print('ValueError', e)
                skip_count += 1
print('skip_count=%d' % skip_count)

./data/news/2010-05-03/
./data/news/2010-05-04/
./data/news/2010-05-05/
./data/news/2010-05-06/
./data/news/2010-05-12/
./data/news/2010-05-14/
./data/news/2010-05-16/
./data/news/2010-05-18/
./data/news/2010-05-19/
./data/news/2010-05-20/
./data/news/2010-05-25/
./data/news/2010-05-26/
./data/news/2010-05-27/
./data/news/2010-05-30/
./data/news/2010-06-01/
./data/news/2010-06-03/
./data/news/2010-06-04/
./data/news/2010-06-07/
./data/news/2010-06-09/
./data/news/2010-06-10/
./data/news/2010-06-11/
./data/news/2010-06-12/
./data/news/2010-06-13/
./data/news/2010-06-14/
./data/news/2010-06-15/
./data/news/2010-06-16/
./data/news/2010-06-17/
./data/news/2010-06-18/
./data/news/2010-06-19/
./data/news/2010-06-20/
./data/news/2010-06-21/
./data/news/2010-06-22/
./data/news/2010-06-23/
./data/news/2010-06-24/
./data/news/2010-06-25/
./data/news/2010-06-26/
./data/news/2010-06-27/
./data/news/2010-06-28/
./data/news/2010-06-29/
./data/news/2010-06-30/
./data/news/2010-07-01/
./data/news/2010

./data/news/2011-05-13/
./data/news/2011-05-14/
./data/news/2011-05-15/
./data/news/2011-05-16/
./data/news/2011-05-17/
./data/news/2011-05-18/
./data/news/2011-05-19/
./data/news/2011-05-20/
./data/news/2011-05-21/
./data/news/2011-05-22/
./data/news/2011-05-23/
./data/news/2011-05-24/
./data/news/2011-05-25/
./data/news/2011-05-26/
./data/news/2011-05-27/
./data/news/2011-05-28/
./data/news/2011-05-29/
./data/news/2011-05-30/
./data/news/2011-05-31/
./data/news/2011-06-01/
./data/news/2011-06-02/
./data/news/2011-06-03/
./data/news/2011-06-04/
./data/news/2011-06-05/
./data/news/2011-06-06/
./data/news/2011-06-07/
./data/news/2011-06-08/
./data/news/2011-06-09/
./data/news/2011-06-10/
./data/news/2011-06-11/
./data/news/2011-06-12/
./data/news/2011-06-13/
./data/news/2011-06-14/
./data/news/2011-06-15/
./data/news/2011-06-16/
./data/news/2011-06-17/
./data/news/2011-06-18/
./data/news/2011-06-19/
./data/news/2011-06-20/
./data/news/2011-06-21/
./data/news/2011-06-22/
./data/news/2011

./data/news/2012-04-27/
./data/news/2012-04-28/
./data/news/2012-04-29/
./data/news/2012-04-30/
./data/news/2012-05-01/
./data/news/2012-05-02/
./data/news/2012-05-03/
./data/news/2012-05-04/
./data/news/2012-05-05/
./data/news/2012-05-06/
./data/news/2012-05-07/
./data/news/2012-05-08/
./data/news/2012-05-09/
./data/news/2012-05-10/
./data/news/2012-05-11/
./data/news/2012-05-12/
./data/news/2012-05-13/
./data/news/2012-05-14/
./data/news/2012-05-15/
./data/news/2012-05-16/
./data/news/2012-05-17/
./data/news/2012-05-18/
./data/news/2012-05-19/
./data/news/2012-05-20/
./data/news/2012-05-21/
./data/news/2012-05-22/
./data/news/2012-05-23/
./data/news/2012-05-24/
./data/news/2012-05-25/
./data/news/2012-05-26/
./data/news/2012-05-27/
./data/news/2012-05-28/
./data/news/2012-05-29/
./data/news/2012-05-30/
./data/news/2012-05-31/
./data/news/2012-06-01/
./data/news/2012-06-02/
./data/news/2012-06-03/
./data/news/2012-06-04/
./data/news/2012-06-05/
./data/news/2012-06-06/
./data/news/2012

./data/news/2013-04-04/
./data/news/2013-04-05/
./data/news/2013-04-06/
./data/news/2013-04-07/
./data/news/2013-04-08/
./data/news/2013-04-09/
./data/news/2013-04-10/
./data/news/2013-04-11/
./data/news/2013-04-12/
./data/news/2013-04-13/
./data/news/2013-04-14/
./data/news/2013-04-15/
./data/news/2013-04-16/
./data/news/2013-04-17/
./data/news/2013-04-18/
./data/news/2013-04-19/
./data/news/2013-04-20/
./data/news/2013-04-21/
./data/news/2013-04-22/
./data/news/2013-04-23/
./data/news/2013-04-24/
./data/news/2013-04-25/
./data/news/2013-04-26/
./data/news/2013-04-27/
./data/news/2013-04-28/
./data/news/2013-04-29/
./data/news/2013-04-30/
./data/news/2013-05-01/
./data/news/2013-05-02/
./data/news/2013-05-03/
./data/news/2013-05-04/
./data/news/2013-05-05/
./data/news/2013-05-06/
./data/news/2013-05-07/
./data/news/2013-05-08/
./data/news/2013-05-09/
./data/news/2013-05-10/
./data/news/2013-05-11/
./data/news/2013-05-12/
./data/news/2013-05-13/
./data/news/2013-05-14/
./data/news/2013

In [11]:
train_df = pd.DataFrame(train_list, columns=["ari_time", "ari_title", "price"])
print("Shape of training data frames:", train_df.shape)

print(len(train_list))
# Print data for checking
for i in range(5):
    print(train_list[i][0], ':::', train_list[i][1])
    print(list(ostock_data_dict.keys())[i])

Shape of training data frames: (312217, 3)
312217
2010-11-19 ::: Shaw Communications Founder and CEO to Step Down Two Months Ahead of Plan
2000-01-03
2010-07-29 ::: Vale Second-Quarter Profit Quadruples to $3.71 Billion From $790 Million
2000-01-04
2010-06-15 ::: ThyssenKrupp to Reduce Steel Processing Next Quarter as Work Done at Mills
2000-01-05
2010-06-16 ::: Industrial Production in U.S. Rose 1.2% in May, Led by Autos
2000-01-06
2010-07-12 ::: European Banks Poised to Win Reprieve on Capital Rules
2000-01-07


In [48]:
tran_df_sort = train_df.sort_values(['ari_time', 'ari_title'], ascending=[1, 1],inplace=True)

In [57]:
train_df[120:130]

,ari_time,ari_title,price
67,2010-06-13,UGB to Sell Tunis Bank Stake to Burgan as Part...,1
52,2010-06-13,Yen Falls Versus Euro on Signs Global Economic...,1
76,2010-06-13,Zimbabwe `Blood Diamonds' Should Be Banned Ove...,1
537,2010-06-14,A2A Chairman Zuccoli Says Mediobanca Advising ...,0
697,2010-06-14,ABN Amro Boosts Covered Bond Issuance to $26 B...,0
347,2010-06-14,AES Unit Enters Wind-Power Agreement With Poli...,0
421,2010-06-14,AIG Rescue Spared European Banks From Raising ...,0
229,2010-06-14,AIG to Be Final Insurer on U.S. Aid as Lincoln...,0
458,2010-06-14,AT&T Apologizes to IPad 3G Users for Security ...,0
541,2010-06-14,Aeroportuario Pacifico Falls After Mexican Exc...,0


In [60]:
%%time
train_df.to_pickle('train.pkl')

CPU times: user 242 ms, sys: 80.5 ms, total: 323 ms
Wall time: 320 ms


In [61]:
%time

# Read saved pickle file for checking

pickle_df = pd.read_pickle('train.pkl')
# train_df.equals(pickle_df)

CPU times: user 12 µs, sys: 0 ns, total: 12 µs
Wall time: 24.6 µs


In [62]:
print(pickle_df.shape)
print(type(pickle_df))
print(pickle_df[:30]["ari_time"])
print(pickle_df[:30]["price"])

(312217, 3)
<class 'pandas.core.frame.DataFrame'>
139    2010-06-13
145    2010-06-13
47     2010-06-13
188    2010-06-13
174    2010-06-13
189    2010-06-13
185    2010-06-13
129    2010-06-13
163    2010-06-13
63     2010-06-13
151    2010-06-13
74     2010-06-13
122    2010-06-13
133    2010-06-13
80     2010-06-13
119    2010-06-13
150    2010-06-13
164    2010-06-13
72     2010-06-13
66     2010-06-13
60     2010-06-13
117    2010-06-13
179    2010-06-13
182    2010-06-13
159    2010-06-13
161    2010-06-13
103    2010-06-13
92     2010-06-13
88     2010-06-13
100    2010-06-13
Name: ari_time, dtype: object
139    1
145    1
47     1
188    1
174    1
189    1
185    1
129    1
163    1
63     1
151    1
74     1
122    1
133    1
80     1
119    1
150    1
164    1
72     1
66     1
60     1
117    1
179    1
182    1
159    1
161    1
103    1
92     1
88     1
100    1
Name: price, dtype: int64


In [65]:
print(pickle_df[120:130])

       ari_time                                          ari_title  price
67   2010-06-13  UGB to Sell Tunis Bank Stake to Burgan as Part...      1
52   2010-06-13  Yen Falls Versus Euro on Signs Global Economic...      1
76   2010-06-13  Zimbabwe `Blood Diamonds' Should Be Banned Ove...      1
537  2010-06-14  A2A Chairman Zuccoli Says Mediobanca Advising ...      0
697  2010-06-14  ABN Amro Boosts Covered Bond Issuance to $26 B...      0
347  2010-06-14  AES Unit Enters Wind-Power Agreement With Poli...      0
421  2010-06-14  AIG Rescue Spared European Banks From Raising ...      0
229  2010-06-14  AIG to Be Final Insurer on U.S. Aid as Lincoln...      0
458  2010-06-14  AT&T Apologizes to IPad 3G Users for Security ...      0
541  2010-06-14  Aeroportuario Pacifico Falls After Mexican Exc...      0
